# 📊 Análise de Dados: Projeto Clima+ 200 (Jaime Basso)

Este *notebook* realiza a **Análise Exploratória de Dados (EDA)** e a visualização dos dados sumarizados. O foco é correlacionar as **leituras diárias dos sensores IoT (Amônia, Temperatura, Umidade, etc.)** com as métricas de desempenho zootécnico dos lotes, testando a eficiência do novo modelo de reaproveitamento de calor e ambiência, conforme o escopo estratégico do projeto. 🐔📈

Os dados utilizados foram gerados a partir do processamento do banco de dados `clima.db` e estão prontos para análise.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Configuração inicial para melhor visualização dos gráficos
sns.set_theme(style="whitegrid", palette="viridis")

# 📂 Definição de Paths (Usando pathlib para um código mais "pythonico")
from pathlib import Path

# Assumindo que este notebook está em 'src' e os dados em '../data/processed'
project_root = Path(os.getcwd()).parent.parent if 'src' in os.getcwd().split(os.sep) else Path(os.getcwd()).parent
processed_data_directory = project_root / 'data' / 'processed'

# 📥 Carregar dados sumarizados
try:
    daily_iot_df = pd.read_csv(processed_data_directory / 'daily_iot_summary.csv')
    lote_performance_df = pd.read_csv(processed_data_directory / 'lote_performance_summary.csv')

    print("✅ Dados IoT Diários Carregados:")
    print(daily_iot_df.head().to_markdown(index=False))
    
    print("\n✅ Desempenho dos Lotes Carregado:")
    print(lote_performance_df.head().to_markdown(index=False))
    
except FileNotFoundError as e:
    print(f"❌ ERRO: Arquivo não encontrado. Verifique o path: {e.filename}")
    # Encerrar ou criar DataFrames vazios para evitar erro no resto do notebook
    daily_iot_df = pd.DataFrame()
    lote_performance_df = pd.DataFrame()

--- 
## 🔍 Análise Exploratória de Dados (EDA) e Tipos de Dados

Verificar a estrutura, valores faltantes e estatísticas básicas é crucial antes da análise principal.

In [ ]:
if not daily_iot_df.empty:
    print("\n✨ Daily IoT Summary Info:")
    daily_iot_df.info()
    print("\n📈 Daily IoT Summary Describe:")
    print(daily_iot_df.describe().T.to_markdown())

if not lote_performance_df.empty:
    print("\n✨ Lote Performance Summary Info:")
    lote_performance_df.info()
    print("\n📈 Lote Performance Summary Describe:")
    print(lote_performance_df.describe().T.to_markdown())

--- 
## 🧹 Limpeza e Pré-processamento de Dados

Converter colunas de **data** para o tipo *datetime* é essencial para manipulação e visualização de séries temporais.

In [ ]:
if not daily_iot_df.empty:
    # Assumindo que o formato '%d/%m/%Y' (dia/mês/ano) está correto.
    daily_iot_df['Data'] = pd.to_datetime(daily_iot_df['Data'], format='%d/%m/%Y', errors='coerce')
    daily_iot_df = daily_iot_df.dropna(subset=['Data']) # Remove linhas com data inválida
    daily_iot_df = daily_iot_df.sort_values(by='Data').reset_index(drop=True)
    print("Data Types after IoT conversion:")
    print(daily_iot_df[['Data']].info())

if not lote_performance_df.empty:
    # Assumindo o mesmo formato para a data de alojamento
    lote_performance_df['data_alojamento'] = pd.to_datetime(lote_performance_df['data_alojamento'], format='%d/%m/%Y', errors='coerce')
    lote_performance_df = lote_performance_df.dropna(subset=['data_alojamento'])
    print("\nData Types after Lote conversion:")
    print(lote_performance_df[['data_alojamento']].info())

--- 
## 🖼️ Visualizações e Análise de Tendências

Visualizações são a melhor forma de encontrar *insights* sobre a ambiência e o desempenho.

In [ ]:
if not daily_iot_df.empty and 'Grandeza' in daily_iot_df.columns:
    # Exemplo 1: Valores diários médios por 'Grandeza' (Série Temporal)
    plt.figure(figsize=(15, 7))
    # Usando style mais limpo do sns
    sns.lineplot(data=daily_iot_df, x='Data', y='average_valor', hue='Grandeza')
    plt.title('Médias Diárias dos Sensores IoT ao Longo do Tempo', fontsize=16)
    plt.xlabel('Data', fontsize=12)
    plt.ylabel('Valor Médio', fontsize=12)
    plt.legend(title='Grandeza', bbox_to_anchor=(1.05, 1), loc=2)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
if not lote_performance_df.empty:
    # Exemplo 2: Desempenho do Lote - Taxa de Mortalidade por Grupo de Teste
    plt.figure(figsize=(10, 6))
    # Usando boxplot para ver a distribuição e outliers
    sns.boxplot(data=lote_performance_df, x='lote_composto', y='mortalidade_percent', hue='teste_realizado')
    plt.title('Distribuição da Mortalidade (%) por Lote e Teste', fontsize=16)
    plt.xlabel('Lote Composto', fontsize=12)
    plt.ylabel('Mortalidade (%)', fontsize=12)
    plt.tight_layout()
    plt.show()

In [ ]:
if not lote_performance_df.empty:
    # Exemplo 3: Matriz de Correlação entre Variáveis Zootécnicas e Ambiência
    
    # Atenção: Lista de colunas duplicadas corrigida e refinada!
    correlation_cols = [
        # Métricas Zootécnicas
        'mortalidade_percent', 'idade_abate', 'conversao_ajustada', 'iep', 'peso_medio',
        'remuneracao_sqr_meter_brl', # Indicador Financeiro
        'aero_parcial', 'condenacao_efetiva_kg', 'contaminacao_parcial', # Indicadores de Qualidade
        # Métricas de Ambiência Agregadas (se existirem na sumarização do lote)
        'avg_umidade', 'avg_temperatura', 'avg_amonia', 'avg_corrente', 'avg_energia', 'avg_pressao'
    ]

    # Filtrar colunas existentes e com dados (sem ser só NaN)
    existing_correlation_cols = [
        col for col in correlation_cols 
        if col in lote_performance_df.columns and not lote_performance_df[col].isnull().all()
    ]

    if len(existing_correlation_cols) > 1:
        plt.figure(figsize=(12, 10))
        # Calcula a correlação e gera o mapa de calor
        corr_matrix = lote_performance_df[existing_correlation_cols].corr()
        sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=.5, linecolor='black')
        plt.title('Matriz de Correlação entre Desempenho e Ambiência', fontsize=16)
        plt.tight_layout()
        plt.show()
    else:
        print("⚠️ Não há colunas suficientes ou relevantes para análise de correlação no DataFrame de Desempenho do Lote.")

--- 
## 💡 Insights e Conclusões (A Ser Preenchido)

Após a execução das células acima, use esta seção para registrar as principais descobertas. 
Foque em: 

1.  **Diferença Amônia/Temperatura:** Há uma correlação clara entre o uso do Clima+ 200 (*teste_realizado*) e os níveis médios de amônia/temperatura? (KPI de qualidade do ar)
2.  **Impacto Zootécnico:** Qual é a correlação entre as variáveis de ambiência (principalmente **Amônia**) e o **IEP** ou **Conversão Alimentar Ajustada**? (KPI de eficiência/custo)
3.  **Sugestão de Ação:** O que os dados sugerem em relação ao modelo de reaproveitamento de calor? Há um *case* claro para a expansão da tecnologia? (Alinhamento estratégico e ESG)